In [3]:
import pandas as pd
import regex as re
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

import warnings
warnings.filterwarnings('ignore')

Volem deixar la taula STATUS_TOTAL així:

- cryptopunk_number
- owner
- for_sale: Yes/No
- sale_eth: quantiat ethereum sense el signe/None
- eth_factors: K, M i Y
- sale_dolars: quantiat dolars sense el signe/None
- dolars factors: B
- are_bids: Yes/No
- bids_eth: quantiat ethereum sense el signe/None
- bids_eth_factors: K, M i Y
- bids_dolars: quantiat dolars sense el signe/None
- bids_dolars_factors: B
- bid_placer: usuari que fa la puja/None

In [2]:
df_1 = pd.read_csv('STATUS_TOTAL.csv')

In [3]:
df_1

,Unnamed: 0,owner,for_sale,are_bids,cryptopunk_number
0,0,This punk is currently owned by address 0xe08c32.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,0
1,1,This punk is currently owned by address 0xb88f61.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,1
2,2,This punk is currently owned by address 0x897aea.,This punk has not been listed for sale by its ...,There is a bid of <0.01Ξ ($4) for this punk fr...,2
3,3,This punk is currently owned by address 0xc352b5.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,3
4,4,This punk is currently owned by address 0xc352b5.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,4
...,...,...,...,...,...
9995,9995,This punk is currently owned by address 0x758353.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,9995
9996,9996,This punk is currently owned by address 0xa9bfa9.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,9996
9997,9997,This punk is currently owned by address 0x279679.,This punk is currently for sale by owner for 4...,There are currently no bids on this punk.,9997
9998,9998,This punk is currently owned by address 0xcfd9cc.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,9998


Eliminem la primera columna, ja que està repetida i canviem l'ordre de les columnes, posant l'id davant:

In [4]:
df_1.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df_1 = df_1[['cryptopunk_number','owner', 'for_sale', 'are_bids']]

Netejem la columna owner perquè ens quedi només l'identificador de l'usuari:

In [6]:
df_1['owner'].replace({'This punk is currently owned by address':''}, regex=True, inplace=True)

In [7]:
df_1['owner'].replace({'\.':''}, regex=True, inplace=True)

Netegem la columna 'for_sale'. Posem 'No' a tots els registres que no han estat en venta i deixem només el preu dels que han estat venuts:

In [8]:
df_1['for_sale'].value_counts()

This punk has not been listed for sale by its owner.               8079
This punk is currently for sale by owner for 140Ξ ($293,839).       190
This punk is currently for sale by owner for 38.95Ξ ($83,588).      155
This punk is currently for sale by owner for 125Ξ ($274,111).        59
This punk is currently for sale by owner for 34.47Ξ ($73,073).       26
                                                                   ... 
This punk is currently for sale by owner for 55.55Ξ ($122,584).       1
This punk is currently for sale by owner for 25.49Ξ ($57,837).        1
This punk is currently for sale by owner for 1KΞ ($2.17M).            1
This punk is currently for sale by owner for 94.49Ξ ($208,511).       1
This punk is currently for sale by owner for 2.25KΞ ($4.77M).         1
Name: for_sale, Length: 1119, dtype: int64

In [9]:
df_1['for_sale'].replace({'This punk has not been listed for sale by its owner.':'No'}, regex=True, inplace=True)

In [10]:
df_1['for_sale'].replace({'This punk is currently for sale by owner for ':''}, regex=True, inplace=True)

In [11]:
df_1['for_sale'].replace({'\.':''}, regex=True, inplace=True)

Separem els preus en dolars i ethereum en dues columnes noves:

In [12]:
df_1[['price_eth','price_dolar']] = df_1['for_sale'].str.split(expand=True)

In [13]:
df_1['for_sale'].replace({'[^"No"]+':'Yes'}, regex=True, inplace=True)

In [14]:
df_1['for_sale'].value_counts()

No     8079
Yes    1921
Name: for_sale, dtype: int64

In [15]:
df_1['for_sale'].value_counts()

No     8079
Yes    1921
Name: for_sale, dtype: int64

Netegem els registres de Bids:

In [16]:
df_1['are_bids'].replace({'There are currently no bids on this punk.':'No'}, regex=True, inplace=True)

In [17]:
df_1['are_bids'].replace({'There is a bid of ':''}, regex=True, inplace=True)

In [18]:
df_1['are_bids']

0                                              No
1                                              No
2        <0.01Ξ ($4) for this punk from 0x7f6498.
3                                              No
4                                              No
                          ...                    
9995                                           No
9996                                           No
9997                                           No
9998                                           No
9999    0.22Ξ ($489) for this punk from 0x20ed17.
Name: are_bids, Length: 10000, dtype: object

Creem les columnes necessaries per incloure tota la informació de bids:

In [19]:
df_1['bids_price'] = df_1['are_bids']

In [20]:
df_1['bids_placer'] = df_1['are_bids']

In [21]:
df_1.loc[df_1['are_bids']!='No', 'are_bids'] = df_1.loc[df_1['are_bids']!='No']['are_bids'].replace({'\w':'', '\W':''}, regex=True)

In [22]:
df_1['are_bids'].replace({'':'Yes'}, regex=True, inplace=True)

In [23]:
df_1['are_bids'].value_counts()

No     9108
Yes     892
Name: are_bids, dtype: int64

Netegem 'price_eth': FALTA SABER QUÈ FER AMB ELS FACTORIALS 'K', 'M' i 'Y' !

In [24]:
df_1['price_eth'].replace({'Ξ':''}, regex=True, inplace=True)

In [25]:
df_1['price_eth'].replace({'No':'None'}, regex=True, inplace=True)

Netegem 'price_dolar':

In [26]:
df_1['price_dolar'].replace({'\(':'', '\)':'', '\$':'', ',':''}, regex=True, inplace=True)

In [27]:
df_1.price_dolar.value_counts()

293839     190
83588      155
274111      59
73073       26
49918       23
          ... 
214837       1
53995        1
146505       1
228029Y      1
58742        1
Name: price_dolar, Length: 1107, dtype: int64

In [28]:
df_1['dolar_factors'] = df_1['price_dolar'].str.extract(r'([a-zA-Z])')

In [29]:
df_1['eth_factors'] = df_1['price_eth'].str.extract(r'([a-zA-Z])')

In [30]:
df_1['eth_factors'].value_counts()

N    8079
K      44
Y      20
M       1
Name: eth_factors, dtype: int64

In [31]:
df_1['dolar_factors'].value_counts()

M    114
Y     20
B      1
Name: dolar_factors, dtype: int64

Separem la columna de de bids en si hi ha bid o no, l'usuari que l'ha fet, el preu en dolars i el preu en ethereum:

In [32]:
df_1['bids_price'].replace({'<':''}, regex=True, inplace=True)

In [35]:
df_1['bids_dolars'] = df_1['bids_price'].str.extract(r'(\$\d+(\,\d+)?[a-zA-Z]?)')

In [36]:
df_1

,cryptopunk_number,owner,for_sale,are_bids,price_eth,price_dolar,bids_price,bids_placer,dolar_factors,eth_factors,bids_dolars
0,0,0xe08c32,No,No,None,None,No,No,NaN,N,NaN
1,1,0xb88f61,No,No,None,None,No,No,NaN,N,NaN
2,2,0x897aea,No,Yes,None,None,0.01Ξ ($4) for this punk from 0x7f6498.,<0.01Ξ ($4) for this punk from 0x7f6498.,NaN,N,$4
3,3,0xc352b5,No,No,None,None,No,No,NaN,N,NaN
4,4,0xc352b5,No,No,None,None,No,No,NaN,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0x758353,No,No,None,None,No,No,NaN,N,NaN
9996,9996,0xa9bfa9,No,No,None,None,No,No,NaN,N,NaN
9997,9997,0x279679,Yes,No,450,991742,No,No,NaN,NaN,NaN
9998,9998,0xcfd9cc,No,No,None,None,No,No,NaN,N,NaN


In [37]:
df_1['bids_dolars'].replace({'\$':''}, regex=True, inplace=True)

In [38]:
df_1['bids_eth'] = df_1['bids_price'].str[:6]

In [39]:
df_1['bids_eth'].head(200)

0          No
1          No
2      0.01Ξ 
3          No
4          No
5      4.20Ξ 
6          No
7          No
8          No
9          No
10         No
11         No
12         No
13         No
14         No
15         No
16         No
17     0.10Ξ 
18         No
19         No
20         No
21         No
22         No
23         No
24         No
25         No
26         No
27         No
28         No
29         No
30         No
31         No
32         No
33         No
34         No
35         No
36         No
37         No
38         No
39     0.02Ξ 
40         No
41         No
42         No
43         No
44         No
45         No
46         No
47         No
48         No
49     0.01Ξ 
50         No
51     0.15Ξ 
52     0.03Ξ 
53         No
54         No
55         No
56         No
57         No
58         No
59         No
60     0.01Ξ 
61     0.25Ξ 
62         No
63     0.01Ξ 
64         No
65         No
66         No
67         No
68         No
69         No
70         No
71    

In [40]:
df_1['bids_eth'].replace({'Ξ':'', 'No':'', '\(\$\d?': ''}, regex=True, inplace=True)

In [41]:
df_1['bids_placer'] = df_1['bids_price'].str.extract(r'(from\s\w+)')

In [42]:
df_1['bids_placer'].replace({'from':''}, regex=True, inplace=True)

In [43]:
df_1.drop('bids_price', axis=1, inplace=True)

In [44]:
factors = {'K': 10**3,
          'M': 10**6,
          'G': 10**9,
          'T': 10**12,
          'P': 10**15,
          'E': 10**18,
          'Z': 10**21,
          'Y': 10**24,
          'B': 10**9,
          }

df_1['eth_factors'] = df_1['eth_factors'].map(factors).astype(float)
df_1['dolar_factors'] = df_1['dolar_factors'].map(factors).astype(float)

In [45]:
df_1.columns

Index(['cryptopunk_number', 'owner', 'for_sale', 'are_bids', 'price_eth', 'price_dolar', 'bids_placer', 'dolar_factors', 'eth_factors', 'bids_dolars', 'bids_eth'], dtype='object')

In [46]:
df_1 = df_1[['cryptopunk_number', 'owner', 'for_sale', 'price_dolar', 'dolar_factors', 'price_eth', 'eth_factors', 
             'are_bids', 'bids_placer', 'bids_dolars', 'bids_eth']]

In [47]:
df_1.replace(r'', np.nan, regex=True, inplace=True)

In [48]:
df_1 = df_1.fillna(value=np.nan)

In [49]:
df_1

,cryptopunk_number,owner,for_sale,price_dolar,dolar_factors,price_eth,eth_factors,are_bids,bids_placer,bids_dolars,bids_eth
0,0,0xe08c32,No,NaN,NaN,None,NaN,No,NaN,NaN,NaN
1,1,0xb88f61,No,NaN,NaN,None,NaN,No,NaN,NaN,NaN
2,2,0x897aea,No,NaN,NaN,None,NaN,Yes,0x7f6498,4,0.01
3,3,0xc352b5,No,NaN,NaN,None,NaN,No,NaN,NaN,NaN
4,4,0xc352b5,No,NaN,NaN,None,NaN,No,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0x758353,No,NaN,NaN,None,NaN,No,NaN,NaN,NaN
9996,9996,0xa9bfa9,No,NaN,NaN,None,NaN,No,NaN,NaN,NaN
9997,9997,0x279679,Yes,991742,NaN,450,NaN,No,NaN,NaN,NaN
9998,9998,0xcfd9cc,No,NaN,NaN,None,NaN,No,NaN,NaN,NaN


In [50]:
df_1.to_csv('estat_punk_actual_df.csv')

In [51]:
df_1['eth_factors'].value_counts()

1.000000e+03    44
1.000000e+24    20
1.000000e+06     1
Name: eth_factors, dtype: int64

In [52]:
df_1['dolar_factors'].value_counts()

1.000000e+06    114
1.000000e+24     20
1.000000e+09      1
Name: dolar_factors, dtype: int64

### TXN_TOTAL

Volem deixar la taula TXN_TOTAL així:

- cryptopunk_number
- txn_type
- from
- to
- amount_dolars
- dolars_factors:
- amount_ethereum
- ethereum_factors:
- txn_date


In [122]:
df_2 = pd.read_csv('TXN_TOTAL.csv')

In [123]:
df_2

,Unnamed: 0,Unnamed: 0.1,txn_type,from,to,amount,txn_date,cryptopunk_number
0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,1,1,Bid Withdrawn,0x983ace,NaN,250Ξ ($1.03M),"May 10, 2021",0.0
2,2,2,Bid,0x983ace,NaN,"250Ξ ($537,615)","Apr 11, 2021",0.0
3,3,3,Bid Withdrawn,0xd7510a,NaN,"100Ξ ($160,973)","Mar 03, 2021",0.0
4,4,4,Bid,0xd7510a,NaN,"100Ξ ($188,897)","Feb 20, 2021",0.0
...,...,...,...,...,...,...,...,...
133269,133269,70605,Bid,0xa0a59c,NaN,0.25Ξ ($69),"Jul 03, 2017",9999.0
133270,133270,70606,Bid,0x717403,NaN,0.20Ξ ($51),"Jun 26, 2017",9999.0
133271,133271,70607,Bid,ddaavvee,NaN,0.20Ξ ($65),"Jun 23, 2017",9999.0
133272,133272,70608,Bid,0x5b098b,NaN,0.15Ξ ($50),"Jun 23, 2017",9999.0


In [124]:
df_2.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [125]:
df_2.drop(0, inplace=True)

In [126]:
df_2.txn_type.value_counts().head(10)

Offered            43487
Bid                21934
Sold               13848
Bid Withdrawn      12429
Transfer           10687
Claimed            10000
Offer Withdrawn     6534
(Wrap)              2297
(Unwrap)            2058
6927                   1
Name: txn_type, dtype: int64

In [127]:
df_2['from'].value_counts(normalize=True)

Pranksy      0.036220
WrappedC…    0.032536
0x717403     0.032520
Hemba        0.019430
0x7760e0     0.017090
               ...   
0x54e70a     0.000016
0x086f30     0.000016
0x9c6b92     0.000016
0xb9051f     0.000016
0x3e37e1     0.000016
Name: from, Length: 4052, dtype: float64

In [128]:
df_2['to'].value_counts(normalize=True)

WrappedC…    0.058542
0xc352b5     0.027219
Hemba        0.024874
0x00d7c9     0.023549
Pranksy      0.018834
               ...   
0x829a08     0.000025
0x139e58     0.000025
0x7989e7     0.000025
0x83fdb6     0.000025
0xdb9f8b     0.000025
Name: to, Length: 4343, dtype: float64

In [129]:
df_2['amount'].value_counts()

0.05Ξ ($14)          1025
0.05Ξ ($13)           800
<0.01Ξ ($<0.01)       657
0.05Ξ ($16)           569
0.03Ξ ($10)           387
                     ... 
130Ξ ($232,913)         1
28Ξ ($67,100)           1
44.90Ξ ($111,153)       1
24.50Ξ ($56,420)        1
16.50Ξ ($32,307)        1
Name: amount, Length: 52622, dtype: int64

In [130]:
df_2['txn_date'].value_counts()

Jun 23, 2017    10452
May 03, 2021     1635
Mar 28, 2021     1287
Sep 28, 2020     1193
Feb 21, 2021     1117
                ...  
Nov 02, 2018        1
Feb 20, 2018        1
Sep 07, 2019        1
Apr 12, 2018        1
Apr 05, 2020        1
Name: txn_date, Length: 1367, dtype: int64

In [131]:
df_2['cryptopunk_number'].value_counts()

3914.0    111
6704.0    104
1737.0    102
7421.0    102
8951.0     99
         ... 
8496.0      1
2171.0      1
2797.0      1
4981.0      1
5459.0      1
Name: cryptopunk_number, Length: 10000, dtype: int64

In [132]:
df_2.columns

Index(['txn_type', 'from', 'to', 'amount', 'txn_date', 'cryptopunk_number'], dtype='object')

In [133]:
df_2 = df_2[['cryptopunk_number','txn_type', 'from', 'to', 'amount', 'txn_date']]

In [134]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133273 entries, 1 to 133273
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   cryptopunk_number  123274 non-null  float64
 1   txn_type           133273 non-null  object 
 2   from               63253 non-null   object 
 3   to                 39237 non-null   object 
 4   amount             91698 non-null   object 
 5   txn_date           123274 non-null  object 
dtypes: float64(1), object(5)
memory usage: 7.1+ MB


Hi ha un nul per cada cryptopunk:

In [135]:
df_2['cryptopunk_number'].isna().sum()

9999

In [136]:
df_2["txn_date"]= pd.to_datetime(df_2["txn_date"])

In [137]:
df_2['txn_date'].value_counts()

2017-06-23    10452
2021-05-03     1635
2021-03-28     1287
2020-09-28     1193
2021-02-21     1117
              ...  
2017-10-22        1
2019-11-17        1
2020-07-18        1
2018-08-14        1
2018-07-17        1
Name: txn_date, Length: 1367, dtype: int64

In [138]:
df_2.isna().sum()

cryptopunk_number     9999
txn_type                 0
from                 70020
to                   94036
amount               41575
txn_date              9999
dtype: int64

Veiem que al final de cada crypto punk hi ha una fila de barrera on els valors son nulls. Les busquem totes i les eliminem:

In [139]:
i = df_2['cryptopunk_number'].isna()

In [140]:
i = df_2[i].index

In [141]:
df_2.drop(i, inplace=True)

In [142]:
df_2.isna().sum()

cryptopunk_number        0
txn_type                 0
from                 60021
to                   84037
amount               31576
txn_date                 0
dtype: int64

Passem la columna de id dels punks a INT:

In [143]:
df_2['cryptopunk_number'] = df_2['cryptopunk_number'].astype(int)

Comencem amb els regex:

In [144]:
df_2['amount'].replace({'<':''}, regex=True, inplace=True)

Creem les columnes per dolars i Ethereum, omplim amb NaN els espais buits de la columna 'amount':

In [145]:
df_2['amount'].replace(r'^(?!\d).*$', np.nan, regex=True, inplace=True)

In [146]:
df_2

,cryptopunk_number,txn_type,from,to,amount,txn_date
1,0,Bid Withdrawn,0x983ace,NaN,250Ξ ($1.03M),2021-05-10
2,0,Bid,0x983ace,NaN,"250Ξ ($537,615)",2021-04-11
3,0,Bid Withdrawn,0xd7510a,NaN,"100Ξ ($160,973)",2021-03-03
4,0,Bid,0xd7510a,NaN,"100Ξ ($188,897)",2021-02-20
5,0,Bid Withdrawn,natealex,NaN,"69Ξ ($25,532)",2020-09-16
...,...,...,...,...,...,...
133269,9999,Bid,0xa0a59c,NaN,0.25Ξ ($69),2017-07-03
133270,9999,Bid,0x717403,NaN,0.20Ξ ($51),2017-06-26
133271,9999,Bid,ddaavvee,NaN,0.20Ξ ($65),2017-06-23
133272,9999,Bid,0x5b098b,NaN,0.15Ξ ($50),2017-06-23


Columna dolars, agafant des del tercer caràcter de l'string i eliminant allò que ens sobra:

In [113]:
df_2['amount_dolars'] = df_2['amount'].str[3:]

In [114]:
df_2['amount_dolars'].replace({'\.?\d*[a-zA-Z]?Ξ':'','\s':'', '\$':'', '\(':'',  '\)':'', '\,':'', '[a-zA-Z]':''}, regex=True, inplace=True)

Columna d'ethereum, agafant des del setè caràcter de l'string i eliminant allò que ens sobra:

In [115]:
df_2['amount_eth'] = df_2['amount'].str[:7]

In [116]:
df_2['amount_eth'].replace({'Ξ':'','\s':'', '\(':'','\)':'','\$':'','\$\d+?': '', '\,':'', '[a-zA-Z]':''}, regex=True, inplace=True)

In [117]:
df_2

,cryptopunk_number,txn_type,from,to,amount,txn_date,amount_dolars,amount_eth
1,0,Bid Withdrawn,0x983ace,NaN,250Ξ ($1.03M),2021-05-10,1.03,250
2,0,Bid,0x983ace,NaN,"250Ξ ($537,615)",2021-04-11,537615,250
3,0,Bid Withdrawn,0xd7510a,NaN,"100Ξ ($160,973)",2021-03-03,160973,100
4,0,Bid,0xd7510a,NaN,"100Ξ ($188,897)",2021-02-20,188897,100
5,0,Bid Withdrawn,natealex,NaN,"69Ξ ($25,532)",2020-09-16,25532,692
...,...,...,...,...,...,...,...,...
133269,9999,Bid,0xa0a59c,NaN,0.25Ξ ($69),2017-07-03,69,0.25
133270,9999,Bid,0x717403,NaN,0.20Ξ ($51),2017-06-26,51,0.20
133271,9999,Bid,ddaavvee,NaN,0.20Ξ ($65),2017-06-23,65,0.20
133272,9999,Bid,0x5b098b,NaN,0.15Ξ ($50),2017-06-23,50,0.15


Fem les columnes de factors corresponents:

In [147]:
df_2['dolar_factors'] = df_2['amount_dolars'].str.extract(r'([a-zA-Z])')

KeyError: 'amount_dolars'

In [119]:
df_2['dolar_factors'].value_counts()

Series([], Name: dolar_factors, dtype: int64)

In [120]:
df_2['eth_factors'] = df_2['amount_eth'].str.extract(r'([a-zA-Z])')

In [121]:
df_2['eth_factors'].value_counts()

Series([], Name: eth_factors, dtype: int64)

In [ ]:
factors = {'K': 10**3,
          'M': 10**6,
          'G': 10**9,
          'T': 10**12,
          'P': 10**15,
          'E': 10**18,
          'Z': 10**21,
          'Y': 10**24,
          'B': 10**9,
          }

df_2['eth_factors'] = df_2['eth_factors'].map(factors).astype(float)
df_2['dolar_factors'] = df_2['dolar_factors'].map(factors).astype(float)

Revisar tema factors, els mès grans fent value counts, per mirar que no doni uns valors massa grans. Fer també per la TAULA 1.

Convertim les columnes a numèric, per fer-ho hem d'omplir els empty strings amb NaN:

In [82]:
df_2.replace(r'', np.nan, regex=True, inplace=True)

In [83]:
df_2['amount_dolars'] = df_2['amount_dolars'].astype(float)

In [84]:
df_2['amount_eth'].replace(r'^(?!\d).*$', np.nan, regex=True, inplace=True)

In [85]:
df_2['amount_eth'] = df_2['amount_eth'].astype(float)

In [87]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123274 entries, 1 to 133273
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   cryptopunk_number  123274 non-null  int32         
 1   txn_type           123274 non-null  object        
 2   from               63253 non-null   object        
 3   to                 39237 non-null   object        
 4   amount             91698 non-null   object        
 5   txn_date           123274 non-null  datetime64[ns]
 6   amount_dolars      91698 non-null   float64       
 7   amount_eth         91698 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int32(1), object(4)
memory usage: 8.0+ MB


Treiem la columna 'amount' que ens sobre i ho passem a dataframe:

In [ ]:
df_2.drop('amount', 1, inplace=True)

In [ ]:
df_2.to_csv('transaccions_hist_df.csv')

## Df atributs:

In [ ]:
from sklearn.preprocessing import LabelEncoder
df= pd.read_csv(r'C:\Users\ab\Itacademy\Jupyter\Projecte final\CryptoPunks-AA\punks_accesories.csv')
df['accesories2']= df['accesories1'].str.rstrip()

# encoder = LabelEncoder()
# df['idaccesories'] = encoder.fit_transform(df['accesories2'])


In [ ]:
len(df.accesories1.unique())

len(df.accesories2.unique())

In [ ]:
df.accesories2[[72,85]]

In [ ]:
df1= pd.read_csv(r'C:\Users\ab\Itacademy\Jupyter\Projecte final\CryptoPunks-AA\Attibutes.csv')

In [ ]:
len(df1['Unnamed: 0'].unique())

In [ ]:
df1

## Nous datasets per transaccions

### Historic preu ethereum - dolars

URL: https://finance.yahoo.com/quote/ETH-USD/history/

In [249]:
columns = ['Date','Open','High','Low','Close','Adj Close','Volume']

In [250]:
eth = pd.read_csv('ETH-USD.CSV')

In [251]:
eth

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-01,7.982310,8.471230,7.982310,8.172570,8.172570,1.473170e+07
1,2017-01-02,8.170870,8.436330,8.054410,8.378510,8.378510,1.457960e+07
2,2017-01-03,8.374580,9.996800,8.317170,9.725250,9.725250,3.362520e+07
3,2017-01-04,9.709290,11.276400,9.559810,11.251600,11.251600,4.105120e+07
4,2017-01-05,11.286800,11.890900,9.402680,10.254400,10.254400,4.155740e+07
...,...,...,...,...,...,...,...
1641,2021-06-30,2164.216309,2282.989014,2090.760742,2274.547607,2274.547607,2.582806e+10
1642,2021-07-01,2274.397461,2274.397461,2081.081055,2113.605469,2113.605469,2.906170e+10
1643,2021-07-02,2109.892578,2155.596436,2021.824829,2150.040283,2150.040283,3.179621e+10
1644,2021-07-03,2150.834961,2237.567139,2117.590088,2226.114258,2226.114258,1.743336e+10


In [252]:
eth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1646 entries, 0 to 1645
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1646 non-null   object 
 1   Open       1642 non-null   float64
 2   High       1642 non-null   float64
 3   Low        1642 non-null   float64
 4   Close      1642 non-null   float64
 5   Adj Close  1642 non-null   float64
 6   Volume     1642 non-null   float64
dtypes: float64(6), object(1)
memory usage: 90.1+ KB


format datetime : %d%b%Y

In [253]:
eth.isna().sum()

Date         0
Open         4
High         4
Low          4
Close        4
Adj Close    4
Volume       4
dtype: int64

In [254]:
eth['Date'] = pd.to_datetime(eth['Date'], format= '%Y-%m-%d')

In [255]:
eth['Date']

0      2017-01-01
1      2017-01-02
2      2017-01-03
3      2017-01-04
4      2017-01-05
          ...    
1641   2021-06-30
1642   2021-07-01
1643   2021-07-02
1644   2021-07-03
1645   2021-07-04
Name: Date, Length: 1646, dtype: datetime64[ns]

In [256]:
eth.set_index('Date', inplace=True)

In [257]:
eth.sort_index(ascending=False, inplace=True)

In [258]:
eth.isna().sum()

Open         4
High         4
Low          4
Close        4
Adj Close    4
Volume       4
dtype: int64

In [259]:
eth

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-07-04,2228.898682,2342.942627,2191.963135,2334.085449,2334.085449,1.906244e+10
2021-07-03,2150.834961,2237.567139,2117.590088,2226.114258,2226.114258,1.743336e+10
2021-07-02,2109.892578,2155.596436,2021.824829,2150.040283,2150.040283,3.179621e+10
2021-07-01,2274.397461,2274.397461,2081.081055,2113.605469,2113.605469,2.906170e+10
2021-06-30,2164.216309,2282.989014,2090.760742,2274.547607,2274.547607,2.582806e+10
...,...,...,...,...,...,...
2017-01-05,11.286800,11.890900,9.402680,10.254400,10.254400,4.155740e+07
2017-01-04,9.709290,11.276400,9.559810,11.251600,11.251600,4.105120e+07
2017-01-03,8.374580,9.996800,8.317170,9.725250,9.725250,3.362520e+07


In [312]:
eth['Variance'] = eth['High'] - eth['Low']

In [314]:
eth['Variance'].head(100)

Date
2021-07-04     150.979492
2021-07-03     119.977051
2021-07-02     133.771607
2021-07-01     193.316406
2021-06-30     192.228272
2021-06-29     165.998536
2021-06-28     176.189941
2021-06-27     168.712280
2021-06-26     130.620362
2021-06-25     223.359130
2021-06-24     144.907348
2021-06-23     215.958863
2021-06-22     285.559204
2021-06-21     392.278686
2021-06-20     225.420167
2021-06-19     109.530029
2021-06-18     229.886718
2021-06-17     144.874023
2021-06-16     200.278320
2021-06-15     124.076172
2021-06-14     137.044677
2021-06-13     234.727783
2021-06-12     181.469726
2021-06-11     168.545166
2021-06-10     184.556153
2021-06-09     212.872803
2021-06-08     305.299560
2021-06-07     261.189942
2021-06-06     127.278808
2021-06-05     259.251220
2021-06-04     294.528076
2021-06-03     223.570557
2021-06-02     245.990967
2021-06-01     208.576904
2021-05-31     436.349853
2021-05-30     283.353271
2021-05-29     358.447510
2021-05-28     425.001953
2021-05

### Històric punks

In [315]:
df = pd.read_csv('df_trans.csv')

In [316]:
df

,Type,From,To,Amount,Transaction,Number
0,Bid withdrawn,0x983ace,NaN,250.0Ξ,"May 10, 2021 0x0987ef",0
1,Bid,0x983ace,NaN,250.0Ξ,"Apr 11, 2021 0xdbf1a0",0
2,Bid withdrawn,0xd7510a,NaN,100.0Ξ,"Mar 3, 2021 0x580c05",0
3,Bid,0xd7510a,NaN,100.0Ξ,"Feb 21, 2021 0x5377c3",0
4,Bid withdrawn,0x63a9db,NaN,69.0Ξ,"Sep 16, 2020 0xbce1fe",0
...,...,...,...,...,...,...
133786,Bid,0xa0a59c,NaN,0.25Ξ,"Jul 3, 2017 0xee9da1",9999
133787,Bid,0x717403,NaN,0.200000000000000001Ξ,"Jun 27, 2017 0x12f9f7",9999
133788,Bid,0x6301Ad,NaN,0.2Ξ,"Jun 24, 2017 0x205b42",9999
133789,Bid,0x5b098b,NaN,0.154Ξ,"Jun 24, 2017 0xfa448c",9999


In [317]:
df.isna().sum()

Type               0
From           72081
To             95213
Amount         44148
Transaction        0
Number             0
dtype: int64

In [318]:
df['Date'] = df['Transaction'].str[:12]

In [319]:
df['Date'].replace({'\s':'-', '\,':''}, regex=True, inplace=True)

In [320]:
df['Date'] = df['Date'].str.rstrip('-')

In [321]:
df['Date'] = pd.to_datetime(df['Date'], format= '%b-%d-%Y')

In [322]:
df['Amount'].replace({'Ξ':'', '\,':''}, regex=True, inplace=True)

In [323]:
df['Amount'] = df['Amount'].astype(float)

In [324]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133791 entries, 0 to 133790
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Type         133791 non-null  object        
 1   From         61710 non-null   object        
 2   To           38578 non-null   object        
 3   Amount       89643 non-null   float64       
 4   Transaction  133791 non-null  object        
 5   Number       133791 non-null  int64         
 6   Date         133791 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 7.1+ MB


In [325]:
df.set_index('Date', inplace = True)

In [326]:
df.sort_index(ascending=False, inplace=True)

In [327]:
df

,Type,From,To,Amount,Transaction,Number
Date,,,,,,
2021-06-30,Offered,NaN,NaN,26.69,"Jun 30, 2021 0x9e1893",7051
2021-06-30,Bid withdrawn,0x12D715,NaN,17.00,"Jun 30, 2021 0x6feee6",6263
2021-06-30,Offer withdrawn,NaN,NaN,NaN,"Jun 30, 2021 0xf18631",9400
2021-06-30,Offered,NaN,NaN,19.94,"Jun 30, 2021 0xb07068",6493
2021-06-30,Transfer,0x8F8B47,0x63d4a5,NaN,"Jun 30, 2021 0xd09776",6281
...,...,...,...,...,...,...
2017-06-23,Claimed,NaN,0xcc7c33,NaN,"Jun 23, 2017 0x82a4ad",8420
2017-06-23,Claimed,NaN,0x005612,NaN,"Jun 23, 2017 0x82a4ad",8421
2017-06-23,Claimed,NaN,0x7e2d1c,NaN,"Jun 23, 2017 0x6d94da",9999


In [328]:
df['user_transaction'] = df['Transaction'].str[12:]

In [329]:
df['user_transaction'].replace({'\s':''}, regex=True, inplace=True)

Afegim el preu de ethereum en dolars en el moment en que es va fer la transacció ajuntant les taules per la columna 'date':

In [330]:
df = df.merge(eth, on='Date', how='inner', indicator=True)

In [331]:
df

,Type,From,To,Amount,Transaction,Number,user_transaction,Open,High,Low,Close,Adj Close,Volume,Variance,_merge
Date,,,,,,,,,,,,,,,
2021-06-30,Offered,NaN,NaN,26.69,"Jun 30, 2021 0x9e1893",7051,0x9e1893,2164.216309,2282.989014,2090.760742,2274.547607,2274.547607,2.582806e+10,192.228272,both
2021-06-30,Bid withdrawn,0x12D715,NaN,17.00,"Jun 30, 2021 0x6feee6",6263,0x6feee6,2164.216309,2282.989014,2090.760742,2274.547607,2274.547607,2.582806e+10,192.228272,both
2021-06-30,Offer withdrawn,NaN,NaN,NaN,"Jun 30, 2021 0xf18631",9400,0xf18631,2164.216309,2282.989014,2090.760742,2274.547607,2274.547607,2.582806e+10,192.228272,both
2021-06-30,Offered,NaN,NaN,19.94,"Jun 30, 2021 0xb07068",6493,0xb07068,2164.216309,2282.989014,2090.760742,2274.547607,2274.547607,2.582806e+10,192.228272,both
2021-06-30,Transfer,0x8F8B47,0x63d4a5,NaN,"Jun 30, 2021 0xd09776",6281,0xd09776,2164.216309,2282.989014,2090.760742,2274.547607,2274.547607,2.582806e+10,192.228272,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-06-23,Claimed,NaN,0xe4d36b,NaN,"Jun 23, 2017 0x218fdb",3029,0x218fdb,336.484985,348.006012,334.989990,341.739014,341.739014,5.932070e+08,13.016022,both
2017-06-23,Claimed,NaN,0x6f4a2d,NaN,"Jun 23, 2017 0x82a4ad",8419,0x82a4ad,336.484985,348.006012,334.989990,341.739014,341.739014,5.932070e+08,13.016022,both
2017-06-23,Claimed,NaN,0xcc7c33,NaN,"Jun 23, 2017 0x82a4ad",8420,0x82a4ad,336.484985,348.006012,334.989990,341.739014,341.739014,5.932070e+08,13.016022,both


In [332]:
df.isna().sum()

Type                    0
From                72079
To                  95211
Amount              44148
Transaction             0
Number                  0
user_transaction        0
Open                  874
High                  874
Low                   874
Close                 874
Adj Close             874
Volume                874
Variance              874
_merge                  0
dtype: int64

In [333]:
df['Amount_dolars'] = df['Amount']*df['Adj Close']

In [334]:
df['Amount_eth'] = df['Amount']

In [335]:
df.columns

Index(['Type', 'From', 'To', 'Amount', 'Transaction', 'Number', 'user_transaction', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Variance', '_merge', 'Amount_dolars', 'Amount_eth'], dtype='object')

In [336]:
df = df[['Number', 'Type', 'From', 'To', 'Amount_dolars', 'Amount_eth', 'user_transaction', 'Adj Close', 'High', 'Low', 'Variance' ]]

In [337]:
df['Amount_dolars'] = df['Amount_dolars'].round(2)

In [338]:
df['Adj Close'] = df['Adj Close'].round(2)

In [339]:
df

,Number,Type,From,To,Amount_dolars,Amount_eth,user_transaction,Adj Close,High,Low,Variance
Date,,,,,,,,,,,
2021-06-30,7051,Offered,NaN,NaN,60707.68,26.69,0x9e1893,2274.55,2282.989014,2090.760742,192.228272
2021-06-30,6263,Bid withdrawn,0x12D715,NaN,38667.31,17.00,0x6feee6,2274.55,2282.989014,2090.760742,192.228272
2021-06-30,9400,Offer withdrawn,NaN,NaN,NaN,NaN,0xf18631,2274.55,2282.989014,2090.760742,192.228272
2021-06-30,6493,Offered,NaN,NaN,45354.48,19.94,0xb07068,2274.55,2282.989014,2090.760742,192.228272
2021-06-30,6281,Transfer,0x8F8B47,0x63d4a5,NaN,NaN,0xd09776,2274.55,2282.989014,2090.760742,192.228272
...,...,...,...,...,...,...,...,...,...,...,...
2017-06-23,3029,Claimed,NaN,0xe4d36b,NaN,NaN,0x218fdb,341.74,348.006012,334.989990,13.016022
2017-06-23,8419,Claimed,NaN,0x6f4a2d,NaN,NaN,0x82a4ad,341.74,348.006012,334.989990,13.016022
2017-06-23,8420,Claimed,NaN,0xcc7c33,NaN,NaN,0x82a4ad,341.74,348.006012,334.989990,13.016022


In [340]:
df.to_csv('transactions_hist_nou.csv')

In [101]:
# df.drop('Transaction', 1, inplace=True)

## Dataset Rarity

URL: https://0xtycoon.github.io/punk-ranks/

In [142]:
columns = ['ranking', 'punk_image','id', 'minscore', '2nd', 'category_score', 'att_count_score',
            'attributes', 'skin','type', 'total_score']

In [143]:
df = pd.read_csv('ranks.csv', names = columns)

In [144]:
df

,ranking,punk_image,id,minscore,2nd,category_score,att_count_score,attributes,skin,type,total_score
0,1,"<img src=""https://www.larvalabs.com/public/ima...","<a href=""https://www.larvalabs.com/cryptopunks...",0.0078,1.15,30.31,0.01,"Buck Teeth 0.7800,Top Hat 1.1500,Big Beard 1.4...",Mid,Male,0.000200
1,2,"<img src=""https://www.larvalabs.com/public/ima...","<a href=""https://www.larvalabs.com/cryptopunks...",0.0142,24.59,0.09,35.60,"Cowboy Hat 1.4200,Earring 24.5900",Alien,Alien,0.000441
2,3,"<img src=""https://www.larvalabs.com/public/ima...","<a href=""https://www.larvalabs.com/cryptopunks...",0.0175,4.19,0.09,45.01,"Medical Mask 1.7500,Knitted Cap 4.1900,Earring...",Alien,Alien,0.000525
3,4,"<img src=""https://www.larvalabs.com/public/ima...","<a href=""https://www.larvalabs.com/cryptopunks...",0.0351,3.51,0.09,3.33,Cap 3.5100,Alien,Alien,0.000677
4,5,"<img src=""https://www.larvalabs.com/public/ima...","<a href=""https://www.larvalabs.com/cryptopunks...",0.0254,3.17,0.09,45.01,"Cap Forward 2.5400,Pipe 3.1700,Small Shades 3....",Alien,Alien,0.000708
...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,"<img src=""https://www.larvalabs.com/public/ima...","<a href=""https://www.larvalabs.com/cryptopunks...",0.2459,24.59,28.24,3.33,Earring 24.5900,Dark,Male,0.681446
9996,9997,"<img src=""https://www.larvalabs.com/public/ima...","<a href=""https://www.larvalabs.com/cryptopunks...",0.2459,24.59,28.24,3.33,Earring 24.5900,Dark,Female,0.681446
9997,9998,"<img src=""https://www.larvalabs.com/public/ima...","<a href=""https://www.larvalabs.com/cryptopunks...",0.2459,24.59,30.06,3.33,Earring 24.5900,Light,Male,0.681778
9998,9999,"<img src=""https://www.larvalabs.com/public/ima...","<a href=""https://www.larvalabs.com/cryptopunks...",0.2459,24.59,30.06,3.33,Earring 24.5900,Light,Female,0.681778


In [145]:
df['id'][0]

'<a href="https://www.larvalabs.com/cryptopunks/details/8348">8348</a>'

In [146]:
df['id'] = df['id'].str.extract(r'(>\d+<)')

In [148]:
df['id'].replace({'<':'', '>':''}, regex=True, inplace=True)

In [150]:
df = df[['id', 'total_score']]

In [151]:
df

,id,total_score
0,8348,0.000200
1,3443,0.000441
2,7523,0.000525
3,2890,0.000677
4,7804,0.000708
...,...,...
9995,4422,0.681446
9996,2578,0.681446
9997,2414,0.681778
9998,5171,0.681778


In [153]:
df.set_index('id', inplace=True)

In [154]:
df

,total_score
id,
8348,0.000200
3443,0.000441
7523,0.000525
2890,0.000677
7804,0.000708
...,...
4422,0.681446
2578,0.681446
2414,0.681778


In [155]:
df.to_csv('rarity_score.csv')